In [ ]:
!pip install spacy

In [2]:
!pip install num2words

  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13724 sha256=c67345b562b0d0187dc721b6ed41d65ee5572a35a83d86b70ee2ed1c1238e92d
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\70\4a\46\1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [3]:
import pandas as pd
import spacy
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
import contractions
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import re
from nltk.tokenize import TweetTokenizer
import regex
import nltk
from nltk.stem import SnowballStemmer
import demoji
import emoji
from sklearn.metrics import f1_score
demoji.download_codes()
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report,accuracy_score
from sklearn.svm import LinearSVC
from num2words import num2words

C:\Users\User\AppData\Local\Temp/ipykernel_10172/424883077.py:17: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [4]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('whitespace')
nltk.download('wordnet')
nltk.download('omw-1.4')
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading whitespace: Package 'whitespace' not found
[nltk_data]     in index
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
data = pd.read_csv('Training.tsv',sep='\t')

In [6]:
data.head(10)

,tweet_id,text,label
0,1382343793341575169,@IrvineWelsh I don’t know about you Irvine but...,0
1,1377631738692796417,I bet money if i went n took a covid test righ...,0
2,1386448010029240326,@JamesMelville My wife received a POSITIVE Cov...,0
3,1361342676340211717,"Out of the 180,000+ people who have had the tw...",0
4,1386757983254765569,My whole family is sick af and here I am now i...,0
5,1382001700853125122,@renfrew1962 @PeakePolly @J_Deliciouso I'm not...,0
6,1383272654212272136,"Test came back positive, no surprise. I have c...",1
7,1374479299047084035,My Pawpaw has been in the hospital a few days....,0
8,1354020426620547072,@MattHancock 4 people I know had covid and rec...,0
9,1362671045136809985,I’m going to sound like I have lost my marbles...,1


In [7]:
print(data.columns)

Index(['tweet_id', 'text', 'label'], dtype='object')


In [8]:
data['label'].value_counts()

label
0    6266
1    1334
Name: count, dtype: int64

# Preprocessing

In [9]:
#emot_object = emot.core.emot()
ps =PorterStemmer()
lemmatiser = WordNetLemmatizer()
english_stopwords = stopwords.words('english')
exclude = set(string.punctuation)
def preprocess(text):
  #text=demoji.findall(df['Text'])
  text = contractions.fix(text.lower(), slang=True)
  #text= re.sub(r'\d+', '', text)
  text = re.sub(r'\d+', lambda x: num2words(int(x.group(0))), text)
  text=re.sub(r'$', '', text)
  text= re.sub(r'’','', text )  
  text=re.sub('<.*?>','',text)
  text=re.sub(r'http\S+', '', text)
  #text=emoji.demojize(text, delimiters=(" ", " "))
  text = ''.join(ch for ch in text if ch not in exclude)
  tokens = word_tokenize(text)
  #print("Tokens:", tokens)
  text = [t for t in tokens if t not in english_stopwords]
  text = " ".join(text)
  return text

In [10]:
import emoji
#import demoji

#demoji.download_codes()
def emo(text):
  temp=emoji.demojize(text,delimiters=(" "," "))
  temp=temp.replace("_","  ")
  return temp

In [11]:
data['emo']=data["text"].apply(lambda x:emo(x))
data["clean_text"]=data['emo'].apply(lambda X: preprocess(X))
#added_data["clean_text"]

In [12]:
data.head()

,tweet_id,text,label,emo,clean_text
0,1382343793341575169,@IrvineWelsh I don’t know about you Irvine but...,0,@IrvineWelsh I don’t know about you Irvine but...,irvinewelsh know irvine keep told covid exist ...
1,1377631738692796417,I bet money if i went n took a covid test righ...,0,I bet money if i went n took a covid test righ...,bet money went n took covid test right going t...
2,1386448010029240326,@JamesMelville My wife received a POSITIVE Cov...,0,@JamesMelville My wife received a POSITIVE Cov...,jamesmelville wife received positive covid tes...
3,1361342676340211717,"Out of the 180,000+ people who have had the tw...",0,"Out of the 180,000+ people who have had the tw...",one hundred eightyzero people two vaccine shot...
4,1386757983254765569,My whole family is sick af and here I am now i...,0,My whole family is sick af and here I am now i...,whole family sick af hospital heart palpitatio...


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['clean_text'], data['label'], test_size=0.33, random_state=42)

In [14]:
data.tail(30)

,tweet_id,text,label,emo,clean_text
7570,1375556191900798982,"In 12 months, I only know of 3 people who test...",0,"In 12 months, I only know of 3 people who test...",twelve months know three people tested positiv...
7571,1386424325268557832,@anet2111 I live in a state with the populatio...,0,@anet2111 I live in a state with the populatio...,anettwo thousand one hundred eleven live state...
7572,1381642657429082112,"Yupppp, I've seen a few stroke patients in the...",0,"Yupppp, I've seen a few stroke patients in the...",yupppp seen stroke patients fortys fiftys reha...
7573,1377644044197752837,@EvelKneidel Actually my gym costs $10 a month...,0,@EvelKneidel Actually my gym costs $10 a month...,evelkneidel actually gym costs ten month way b...
7574,1375572937739210754,@annabkrr Got my second shot &amp; I continue ...,0,@annabkrr Got my second shot &amp; I continue ...,annabkrr got second shot amp continue wear mas...
7575,1387648123246350338,Went to hospital yesterday because I had some ...,1,Went to hospital yesterday because I had some ...,went hospital yesterday chest pain got chest x...
7576,1387906108514136066,Well... I got my second Pfizer dose two weeken...,0,Well... I got my second Pfizer dose two weeken...,well got second pfizer dose two weekends ago r...
7577,1304095108862586880,"last night, after reading about the 46-year-ol...",0,"last night, after reading about the 46-year-ol...",last night reading fortysixyearold woman diagn...
7578,1373826228105383937,"And a few weeks ago, I had COVID (very mild th...",1,"And a few weeks ago, I had COVID (very mild th...",weeks ago covid mild thankfully wife found got...
7579,1349069842377187328,Seeing all these Members of Congress (who’ve a...,0,Seeing all these Members of Congress (who’ve a...,seeing members congress received first dose va...


# Feature Extraction: TF-IDF (char_wb)

In [16]:
Tfidf_vec = TfidfVectorizer(analyzer='char_wb', ngram_range=(1, 5), max_df=1.0, min_df=1, max_features=5000)
count_train = Tfidf_vec.fit(X_train)
train_features1 = Tfidf_vec.transform(X_train)
test_features1 = Tfidf_vec.transform(X_test)

Feature Extraction- (word) TFIDF

In [19]:
 Tfidf_vec1 = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), max_df=1.0, min_df=1, max_features=5000)
 count_train1 = Tfidf_vec1.fit(X_train)
 train_features2= Tfidf_vec1.transform(X_train)
 test_features2 = Tfidf_vec1.transform(X_test)

# Model Building with SVM - LinearSVC

In [20]:
clf1 =LinearSVC(C=1.0, class_weight="balanced", max_iter=10000, random_state=123)
clf1.fit(train_features1, y_train)
y_pred1=clf1.predict(test_features1)
accuracy = accuracy_score(y_test, y_pred1)

print("Test Accuracy:", round(accuracy*100, 4))

print("\n", classification_report(y_test, y_pred1))

Test Accuracy: 79.8644

               precision    recall  f1-score   support

           0       0.92      0.82      0.87      2040
           1       0.47      0.69      0.56       468

    accuracy                           0.80      2508
   macro avg       0.70      0.76      0.72      2508
weighted avg       0.84      0.80      0.81      2508



In [21]:
clf1 =LinearSVC(C=1.0, class_weight="balanced", max_iter=10000, random_state=123)
clf1.fit(train_features2, y_train)
y_pred2=clf1.predict(test_features2)
accuracy = accuracy_score(y_test, y_pred2)

print("Test Accuracy:", round(accuracy*100, 4))

print("\n", classification_report(y_test, y_pred2))

Test Accuracy: 81.7783

               precision    recall  f1-score   support

           0       0.91      0.86      0.88      2040
           1       0.51      0.63      0.57       468

    accuracy                           0.82      2508
   macro avg       0.71      0.75      0.72      2508
weighted avg       0.84      0.82      0.83      2508

